In [1]:
import locale
   ...: locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

import pandas as pd
import pyarrow
import plotly.express as px
import yaml
from IPython.display import display, Markdown
import datetime
import dateutil.relativedelta

#for key, value in locale.localeconv().items():
#    print("%s: %s" % (key, value))

# Suporte a pessoas com Tabagismo

Essa seção se dedica a identificar como é o registro do acompanhamento de pessoas com tabagismo no município.

As pessoas foram computadas se tivessem códigos CID-10, CIAP-2 ou ABP relacionados ao tabaco registrados, ou se tiveram foram registrados para elas procedimentos relacionados a abordagem do fumante. 

Os códigos CID-10 e CIAP-2 utilizados são:  
  
- F17 - Transtornos psíquicos e comportamentais por uso de tabaco  
- Z71.6 - Aconselhamento em relação ao consumo de tabaco  
- Z72.0 - Uso de Tabaco  
- CIAP-2 - P17 Abuso do Tabaco  
- ABP011 - Abordagem do Tabagismo  


In [2]:


display(Markdown("## Dados gerais de perfil do indicador"))

with open('P13_Tabagismo.yml', 'r') as file:
    indicador = yaml.safe_load(file)

#for P in indicador['perfil'].keys():
#    display(Markdown(f"**{P}:** {indicador['perfil'][P]}  "))
#    print(P + ": " + str(indicador['perfil'][P]) + " ")

display(Markdown(f"O indicador é do tipo **{indicador['perfil']['tipo']}**."))

display(Markdown(f"Serão avaliados registros de:"))

for P in indicador['perfil']['fichas']:
    display(Markdown(f"- {P}"))

display(Markdown(f"Realizados nos últimos {indicador['perfil']['meses']} meses."))


## Dados gerais de perfil do indicador

O indicador é do tipo **cobertura proporcional**.

Serão avaliados registros de:

- atendimento individual

Realizados nos últimos 12 meses.

In [3]:


df = pd.read_parquet('Cidade_X/atendimentos.parquet')

display(Markdown("## Dados gerais da população atendida"))

# display(Markdown("### Todo período do banco"))

# Nota: preciso delimitar os últimos 12 meses apenas.

atendimentos_total = df.shape[0]
populacao_total = df['paciente_id'].nunique()

# Deixando o número em português

pt_atendimentos = locale.format_string('%u', atendimentos_total, grouping=True )
pt_populacao = locale.format_string('%u', populacao_total, grouping=True )

# display(Markdown(f"Total de atendimentos = **{pt_atendimentos}**.  "))
# display(Markdown(f"Pessoas atendidas = **{pt_populacao}**.  "))

agora = pd.Timestamp.now()
antiga = df.co_dim_tempo.min()
recente = df.co_dim_tempo.max()

display(Markdown("**Atualização do banco**  "))
display(Markdown(f"Data e horas de hoje: {agora.strftime('%Y-%m-%d')}  "))
display(Markdown(f"Registro mais antigo: {antiga}  "))
display(Markdown(f"Registro mais recente: {recente}  "))

display(Markdown(f"**Período para apuração do indicador:**  {indicador['perfil']['meses']} meses"))


inicio = agora - dateutil.relativedelta.relativedelta(months=int(indicador['perfil']['meses']))



## Dados gerais da população atendida

**Atualização do banco**  

Data e horas de hoje: 2024-01-30  

Registro mais antigo: 2011-01-01  

Registro mais recente: 2024-01-21  

**Período para apuração do indicador:**  12 meses

In [4]:
display(Markdown("### Período de apuração do indicador"))

display(Markdown(f"Serão considerados os atendimentos a partir de **{inicio.strftime('%Y-%m-%d')}**"))

df['co_dim_tempo'] = pd.to_datetime(df['co_dim_tempo'])
df = df.loc[(df['co_dim_tempo'] >= inicio)]

atendimentos_total = df.shape[0]
populacao_total = df['paciente_id'].nunique()

# Deixando o número em português

pt_atendimentos = locale.format_string('%u', atendimentos_total, grouping=True )
pt_populacao = locale.format_string('%u', populacao_total, grouping=True )

display(Markdown(f"Total de atendimentos em {indicador['perfil']['meses']} meses = **{pt_atendimentos}**.  "))
display(Markdown(f"Pessoas atendidas {indicador['perfil']['meses']} meses= **{pt_populacao}**.  "))


### Período de apuração do indicador

Serão considerados os atendimentos a partir de **2023-01-30**

Total de atendimentos em 12 meses = **134.773**.  

Pessoas atendidas 12 meses= **37.358**.  

In [5]:


display(Markdown("## Cálculo do denominador"))

display(Markdown("### Atendimentos codificados com CID-10"))

df1 = pd.DataFrame()
df2 = pd.DataFrame()

for C in indicador['denominador']['filtro_cids']:
    df1 = df[df['ds_filtro_cids'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f"CID {C} encontrados **{N_atend}**  "))
    df2 = pd.concat([df1, df2], ignore_index=True).drop_duplicates()

atd_cid = df2.shape[0]
pessoas_cid = df2['paciente_id'].nunique()

display(Markdown(f"Atendimentos por tabagismo, removendo duplicados = **{atd_cid}**.  "))
display(Markdown(f"Pessoas Tabagistas encontradas via CID-10 = **{pessoas_cid}**.  "))

## Cálculo do denominador

### Atendimentos codificados com CID-10

CID F17 encontrados **30**  

CID Z71.6 encontrados **0**  

CID Z72.0 encontrados **0**  

Atendimentos por tabagismo, removendo duplicados = **30**.  

Pessoas Tabagistas encontradas via CID-10 = **30**.  

In [6]:


display(Markdown("### Atendimentos codificados com CIAP-2 ou ABP"))

df1 = pd.DataFrame()
df3 = pd.DataFrame()


for C in indicador['denominador']['filtro_ciaps']:
    df1 = df[df['ds_filtro_ciaps'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f" {C} encontrados **{N_atend}**  "))
    df3 = pd.concat([df1, df3], ignore_index=True).drop_duplicates()

atd_ciap = df3.shape[0]
pessoas_ciap = df3['paciente_id'].nunique()

display(Markdown(f"Atendimentos por tabagismo, removendo duplicados = **{atd_ciap}**."))
display(Markdown(f"Pessoas Tabagistas encontradas via CID-10 = **{pessoas_ciap}**."))

### Atendimentos codificados com CIAP-2 ou ABP

 P17 encontrados **367**  

 ABP011 encontrados **22**  

Atendimentos por tabagismo, removendo duplicados = **389**.

Pessoas Tabagistas encontradas via CID-10 = **299**.

In [7]:
populacao_alvo = 0

display(Markdown(f"### Total de pessoas para: {indicador['perfil']['nome_indicador']}"))

lista_denominador = pd.concat([df2, df3], ignore_index=True).drop_duplicates()

atd_alvo = lista_denominador.shape[0]
populacao_alvo = lista_denominador['paciente_id'].nunique()

display(Markdown(f"Total de atendimentos pela condição alvo = **{atd_alvo}**.  "))
display(Markdown(f"Total de pessoas na população alvo = **{populacao_alvo}**."))

### Total de pessoas para: Abordagem individual do tabagismo

Total de atendimentos pela condição alvo = **390**.  

Total de pessoas na população alvo = **300**.

In [8]:


import plotly.graph_objects as go

labels = ['População alvo','População total']

values = [populacao_alvo, populacao_total]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=['2775b1', '78b4d1']))
fig.show()

percentual = populacao_alvo / populacao_total * 100

percentual = locale.format_string('%10.2f', percentual)

display(Markdown(f"Sua população-alvo para esse indicador corresponde a {percentual}% da população total."))


Sua população-alvo para esse indicador corresponde a       0,80% da população total.

## Cálculo do Numerador

In [9]:


display(Markdown("### Atendimentos com abordagem individual do tabagismo"))

df1 = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()

display(Markdown("### Perfil da Condição A"))

for P in indicador['condição_A'].keys():
    display(Markdown(f"{P}: {indicador['condição_A'][P]}  "))


### Atendimentos com abordagem individual do tabagismo

### Perfil da Condição A

descrição: Abordagem individual do tabagismo registrado nos atendimentos  

fonte: atendimento individual  

profissional: medico, enfermeiro  

procedimentos_solicitados: None  

procedimentos_avaliados: ['0301010099']  

valor_min: 1  

In [10]:
display(Markdown("### Avaliação da Condição A:"))

df3 = pd.DataFrame()

for C in indicador['condição_A']['procedimentos_avaliados']:
    C = C.replace(".", "")
    C = C.replace("-", "")
    df1 = df[df['ds_filtro_proced_avaliados'].str.contains(C)]
    N_atend = df1.shape[0]
    display(Markdown(f"Procedimentos {C} avaliados **{N_atend}**  "))
    df3 = pd.concat([df1, df3], ignore_index=True).drop_duplicates()

procedimentos = df3.shape[0]
pessoas_beneficiadas = df3['paciente_id'].nunique()

df_pacientes_procedimentos = df3.groupby(['paciente_id']).agg(ds_filtro_proced_avaliados_count=('ds_filtro_proced_avaliados', 'count')).reset_index()
df_condição_A = df_pacientes_procedimentos[df_pacientes_procedimentos.ds_filtro_proced_avaliados_count >= int(indicador['condição_A']['valor_min'])]
pessoas_beneficiadas_A = df_condição_A.shape[0]

display(Markdown(f"Total de atendimentos com abordagem do tabagismo = **{procedimentos}**.  "))
display(Markdown(f"Pessoas beneficiadas com ao menos 1 abordagem do tabagismo = **{pessoas_beneficiadas}**."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


### Avaliação da Condição A:

Procedimentos 0301010099 avaliados **0**  

Total de atendimentos com abordagem do tabagismo = **0**.  

Pessoas beneficiadas com ao menos 1 abordagem do tabagismo = **0**.

In [11]:

display(Markdown("### Cálculo do indicador"))

display(Markdown(f"População-alvo: {populacao_alvo}"))

#df_indicador = pd.merge(df_condição_A, df_condição_B, on='paciente_id')
df_indicador = df_condição_A

pessoas_beneficiadas = df_indicador.shape[0]

Pessoas_não_atendidas = populacao_alvo - pessoas_beneficiadas

labels = ['Pessoas não atendidas', 'Pessoas beneficiadas']
values = [Pessoas_não_atendidas, pessoas_beneficiadas]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
fig.update_traces(marker=dict(colors=['2775b1', '78b4d1']))
fig.show()

if populacao_alvo == 0:
    display(Markdown("### Não foi encontrada população-alvo. Não será possível calcular indicadores"))
    pessoas_beneficiadas = 0
else:
    indicador = pessoas_beneficiadas / populacao_alvo * 100
    indicador = locale.format_string('%10.2f', indicador)


if pessoas_beneficiadas == 0:
    display(Markdown("Nenhuma pessoa recebeu o tratamento preconizado neste indicador"))
else:
    display(Markdown(f"{indicador}% da população analisada recebeu o tratamento preconizado neste indicador."))

# Pensar na fórmula para quando o N necessário for maior do que 1. 


### Cálculo do indicador

População-alvo: 300

Nenhuma pessoa recebeu o tratamento preconizado neste indicador